In [7]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,confusion_matrix,roc_auc_score


## Get the Data and Labels

In [2]:
num_testsub = 32
num_classes = 4
num_channel = 40
num_datapoints = 8064
num_trials = 40
sampling_rate = 128 # 128Hz as given in the data

def loadfiles_normalized():
    data_dict = {}
    print("Loading files into data_dict .................")
    for i in range(num_testsub):
                    if i < 10:
                        name = '%0*d' % (2,i+1)
                    else:
                        name = i+1
                    fname = 'data/data_prepared/data_norm_mischa/noramlized_datasub'+str(name) +'.npy'
                    data_dict["sub%s" %name] = np.load(fname)    
    print("Loaded!!!!!") 
    return data_dict

In [8]:
data_dict = loadfiles_normalized()

Loading files into data_dict .................
Loaded!!!!!


In [3]:
valence = np.load('data/data_prepared/labels/label_class_0.npy')
arousal = np.load('data/data_prepared/labels/label_class_1.npy')
dominance = np.load('data/data_prepared/labels/label_class_2.npy')
liking = np.load('data/data_prepared/labels/label_class_3.npy')

In [4]:
valence.shape

(1280,)

In [11]:
data_dict['sub01'].shape

(240, 40, 9)

In [22]:
sub01 = data_dict['sub01'][0:6]

In [25]:
sub01 = sub01.reshape(240,9)

In [26]:
sub01.shape

(240, 9)

In [32]:
data_dict['sub01'].shape[1]

40

In [ ]:
# 20 participants 240, 9 
train = [][][]

In [89]:
def split_data_into_train_test_valid(data_dict):
    # one video is 6 matrices of shape 40 * 9 -->  240 * 9 
    # one participant has 40 videos --> 240 * 40 --> 9600 , 9

    train = np.empty((192000,9)) # 20 participants --> 9600*20 , 9
    train[:] = np.nan
    valid = np.empty((57600,9)) # 6 participants --> 9600*6 , 9
    valid[:] = np.nan
    test = np.empty((57600,9)) # 6 participants --> 9600*6 , 9
    test[:] = np.nan


    list_of_participants = []
    # loop over all participants
    for sub in data_dict.keys():
        one_participant = np.empty((9600,9))
        one_participant[:] = np.nan
        # loop over all channels (40)
        for channel in range(data_dict[sub].shape[1]):
            temp = data_dict[sub][channel*6:(channel+1)*6]
            temp = temp.reshape(240,9)
            #train[channel*240:(channel+1)*240] = temp
            one_participant[channel*240:(channel+1)*240] = temp
        list_of_participants.append(one_participant)

    j = 0
    k = 0
    for i in range(len(list_of_participants)):
        if i < 20: 
            train[9600*i:9600*(i+1)] = list_of_participants[i]
        if (i >= 20) & (i < 26):
            valid[9600*j:9600*(j+1)] = list_of_participants[i]
            print(9600*j, 9600*(j+1))
            j = j+1
        if i >= 26:
            test[9600*k:9600*(k+1)] = list_of_participants[i]
            k = k+1
    return train, valid, test

In [90]:
train, valid, test = split_data_into_train_test_valid(data_dict)

0 9600
9600 19200
19200 28800
28800 38400
38400 48000
48000 57600


In [91]:
np.isnan(train).any()

False

In [92]:
np.isnan(valid).any()

False

In [93]:
np.isnan(test).any()

False

In [81]:
np.isnan(test)

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [63]:
temp = list_of_participants[0:1]
temp[0].shape

(9600, 9)

In [37]:
for channel in range(data_dict['sub01'].shape[1]):
    print(data_dict['sub01'][channel*6:(channel+1)*6].shape)

(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)
(6, 40, 9)


In [38]:
for channel in range(data_dict['sub01'].shape[1]):
    print(channel)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


In [ ]:
# Define KNN based model for augmented data
def knnmodel(xtrain,ytrain,xtest,ytest,kval):
  
  # print("xtrain.shape,ytrain.shape",xtrain.shape,ytrain.shape)
  # print("xtest.shape,ytest.shape",xtest.shape,ytest.shape)

  # Define the KNN model here

  model = KNeighborsClassifier(n_neighbors=kval)
  model.fit(xtrain, ytrain)
  y_pred = model.predict(xtest)

  # print("Accuracy score for fold",fold_no)
  acc = accuracy_score(ytest, y_pred)*100
  acc = round(acc, 4)
  print("Accuracy",acc)
  print(classification_report(ytest, y_pred))
  print(confusion_matrix(ytest,y_pred))
  # return acc,xtrain, xtest, ytrain, ytest
  return acc